# DMRG code
this is a dmrg to get the ground state energy from tenpy package 

In [129]:
import time as tt
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import tenpy as tp
tp.tools.misc.setup_logging(to_stdout="INFO")
# from numba import njit, prange
# from numba import njit, prange

In [235]:
L = 10
Vv, Jj = 1.0, 0.0

# model_params = {
#     'J': 1. , 'g': 1.,  # critical
#     'L': L,
#     'bc_MPS': 'finite',
#     'bc_x': 'periodic',
# }
# M = tp.models.tf_ising.TFIChain(model_params)

model_paramss = {
    'Jxx': -2*Jj , 'Jz': Vv/8, 'hz':-Vv/4,
    'L': L,
    'bc_x': 'open',
    'bc_MPS': 'finite',
}

dmrg_params = {
    'mixer': False,  # setting this to True helps to escape local minima
    'max_E_err': 1.e-10,
    'trunc_params': {
        'chi_max': 100,
        'svd_min': 1.e-10,
    },
    'verbose': True,
    'combine': True,
};


M = tp.models.xxz_chain.XXZChain(model_paramss);
# M.add_onsite_term(-Vv/8, 0, 'Sz')
# M.add_onsite_term(-Vv/8, L-1, 'Sz')
psi = tp.networks.mps.MPS.from_lat_product_state(M.lat, [['up'],['down']])
eng = tp.algorithms.dmrg.TwoSiteDMRGEngine(psi, M, dmrg_params);
E, psi = eng.run();


INFO    : XXZChain: reading 'L'=10
INFO    : XXZChain: reading 'Jxx'=-0.0
INFO    : XXZChain: reading 'Jz'=0.125
INFO    : XXZChain: reading 'hz'=-0.25
INFO    : XXZChain: reading 'bc_MPS'='finite'
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))

INFO    : TwoSiteDMRGEngine: subconfig 'trunc_params'=Config(<2 options>, 'trunc_params')
INFO    : TwoSiteDMRGEngine: reading 'combine'=True
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))

INFO    : TwoSiteDMRGEngine: reading 'mixer'=False
INFO    : Running sweep with optimization
INFO    : trunc_params: reading 'chi_max'=100
INFO    : trunc_params: reading 'svd_min'=1e-10
INFO    : checkpoint after sweep 1
energy=-0.2812500000000000, max S=-0.0000000000000000, age=10, norm_err=0.0e+00
Current memory usage 108.3MB, wall time: 0.2s
Delta E = nan, Delta S = 0.0000e+00 (per sweep)
max trunc_err = 0.0000e+00, max E_trunc = 0.0000e+00
chi: [1, 1, 1, 1, 1, 1, 1, 1, 1]
INFO    : Running sweep with optimization
INFO

In [238]:
print("ground state energy = ", E )
print("constant energy = ", Vv*(L-1)/8 )

Z = psi.expectation_value('Sz')
print("total magnetization =", np.sum(Z))


ground state energy =  -0.28125
constant energy =  1.125
total magnetization = 0.0


In [65]:
from tenpy.models.lattice import Site, Chain
from tenpy.models.model import CouplingModel, NearestNeighborModel, MPOModel, CouplingMPOModel
from tenpy.linalg import np_conserved as npc
from tenpy.tools.params import asConfig
from tenpy.networks.site import SpinHalfSite

In [ ]:
class SpinFermionChain(CouplingMPOModel, NearestNeighborModel, MPOModel):
    default_lattice = "Chain"
    force_default_lattice = True

    def init_sites(self, model_params):
        sort_charge = model_params.get('sort_charge', True, bool)
        return SpinHalfSite(conserve=None, sort_charge=sort_charge)  # use predefined Site

    def init_terms(self, model_params):
        # read out parameters
        J = model_params.get('J', 1., 'real_or_array')
        V = model_params.get('V', 1., 'real_or_array')
        # Jzz = model_params.get('Jzz', 0., 'real_or_array')
        # add terms
        for u in range(len(self.lat.unit_cell)):
            # print("**********************")
            # print(u,",",",",u)
            # print("**********************")
            self.add_onsite(+V/4, u, 'Sz')
            # self.add_onsite(-hx, u, 'Sx')
        
        self.add_onsite(-V/8, 0, 'Sz')
        self.add_onsite(-V/8, len(self.lat.unit_cell)-1, 'Sz')
        
        # for u1, u2, dx in self.lat.pairs['nearest_neighbors']:
        # for u1, u2, dx in self.lat.nearest_neighbors:
        #     print("**********************")
        #     print(u1,",",u2,",",dx)
        #     print("**********************")
        #     self.add_coupling(-J, u1, 'Sp', u2, 'Sm', dx)
        #     self.add_coupling(-J, u1, 'Sm', u2, 'Sp', dx)
        #     self.add_coupling(V/8, u1, 'Sz', u2, 'Sz', dx)
            

In [116]:
L = 8
Vv, Jj = 0.3, 1.0

model_paramss = {
    'J': -1.0 , 'V': 0.1,
    'L': L,
    'bc_x': 'periodic',
    'bc_MPS': 'finite',
}

dmrg_params = {
    'mixer': True,  # setting this to True helps to escape local minima
    'max_E_err': 1.e-10,
    'trunc_params': {
        'chi_max': 100,
        'svd_min': 1.e-10,
    },
    # 'verbose': True,
    'combine': True,
};


M = SpinFermionChain(model_paramss);
psi = tp.networks.mps.MPS.from_lat_product_state(M.lat, [['up'],['down']])
# eng = tp.algorithms.dmrg.TwoSiteDMRGEngine(psi, M, dmrg_params);
# E, psi = eng.run();


# psi = tp.networks.mps.MPS.from_lat_product_state(M.lat, [['up'],['down']])
# eng = tp.algorithms.dmrg.TwoSiteDMRGEngine(psi, M, dmrg_params);

INFO    : SpinFermionChain: reading 'bc_MPS'='finite'
INFO    : SpinFermionChain: reading 'bc_x'='periodic'
INFO    : SpinFermionChain: reading 'L'=8
INFO    : SpinFermionChain: reading 'J'=-1.0
INFO    : SpinFermionChain: reading 'V'=0.1


IndexError: list index out of range